# Why A₂ and p₄ Have Different Null Distributions

## The observation

Under permutation testing (null model fitting), both A₂ and p₄ have Broderick reference values close to zero, but their null distributions look very different:
- **A₂**: null distribution is narrow, peaking near 0 → actual error falls at ~70th percentile (not significant)
- **p₄**: null distribution is wide, spread far from 0 → actual error falls at ~17th percentile (significant)

This is puzzling because both parameters start near zero. Why does p₄ drift so much further than A₂ when fitting random (permuted) data?

## The answer: two effects work together

1. **Structural effect — inside vs. outside the Gaussian**: A₂ scales the prediction amplitude (outside the Gaussian); p₄ shifts the tuning peak (inside log₂(Pᵥ), inside the Gaussian exponent). This creates a gradient amplification factor for p₄ that A₂ lacks. Even without any normalization, p₄ drifts ~2.5× more than A₂.

2. **L2 normalization amplifies this**: the loss function normalizes predictions by their L2 norm before comparing to data. A₂’s amplitude changes are divided out by the norm → the loss becomes insensitive to A₂ → its gradient collapses to near zero as it moves. p₄’s shape changes survive normalization → the gradient stays nonzero → p₄ keeps drifting. This amplifies the drift ratio from ~2.5× to **~25×**.

This notebook walks through: (1) what a gradient is, (2) the analytical derivation of both effects, and (3) a numerical simulation confirming both.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 11})

## Step 1: What is a gradient?

The **gradient** of the loss $L$ with respect to a parameter $\theta$ answers: *if I nudge $\theta$ slightly, how much does $L$ change?*

$$\frac{\partial L}{\partial \theta} = \lim_{\epsilon \to 0} \frac{L(\theta + \epsilon) - L(\theta)}{\epsilon}$$

The optimizer uses the gradient to update each parameter every training step:
$$\theta_{\text{new}} = \theta_{\text{old}} - \eta \cdot \frac{\partial L}{\partial \theta}$$
where $\eta$ is the learning rate.

**Critical consequence**: if $\partial L / \partial \theta \approx 0$, the update is nearly zero regardless of learning rate. A parameter with a consistently tiny gradient stays near its initialization even after thousands of training steps.

This is precisely why A₂ stays near zero under null fitting: its gradient is tiny, so the optimizer never moves it far.

PyTorch computes gradients automatically via `.backward()`. The cell below confirms this with a toy example.

In [ ]:
# Toy example: L = (x - 3)^2, minimum at x = 3, gradient = 2(x - 3)
x = torch.tensor(0.0, requires_grad=True)
loss = (x - 3) ** 2
loss.backward()  # PyTorch computes dL/dx automatically

x_vals = np.linspace(-1, 7, 200)
L_vals = (x_vals - 3) ** 2

fig, ax = plt.subplots(figsize=(5.5, 3.8))
ax.plot(x_vals, L_vals, 'k-', lw=2, label=r'$L = (x - 3)^2$')
ax.plot(0, 9, 'ro', ms=11, zorder=5, label='current position (x = 0)')
ax.plot(3, 0, 'g*', ms=14, zorder=5, label='minimum (x = 3, gradient = 0)')
ax.annotate('', xy=(1.4, 9), xytext=(0.05, 9),
            arrowprops=dict(arrowstyle='->', color='red', lw=2.5))
ax.text(0.1, 10.2,
        f'gradient = {x.grad.item():.0f}\n'
        r'$\Rightarrow$ move $x$ right toward minimum',
        color='red', fontsize=10)
ax.set_xlabel('x'), ax.set_ylabel('Loss L')
ax.set_title('Gradient = slope of the loss at the current parameter value')
ax.legend(loc='upper right', fontsize=9)
plt.tight_layout()
plt.show()

## Step 2: The model equations and why gradients differ

### The 2D model

The model predicts the BOLD response for each stimulus as:

$$\text{pred} = \underbrace{A_v}_{\text{amplitude}} \cdot \underbrace{\exp\!\left(-\frac{u^2}{2\sigma^2}\right)}_{G \;=\; \text{Gaussian}}, \qquad u = \log_2(\omega_l) + \log_2(P_v)$$

- $\omega_l$ is the stimulus spatial frequency
- $u$ measures how far the stimulus is from the voxel’s tuning peak in log₂-frequency: $u = 0$ at peak, $|u| > 0$ for off-peak stimuli
- $G = \exp(-u^2/2\sigma^2)$ is always in $[0, 1]$

The two modulators are:
$$A_v = 1 + A_1\cos(2\theta_l) + \mathbf{A_2}\cos(4\theta_l) + \ldots \qquad \text{\textbf{A₂ lives here — outside the Gaussian}}$$

$$P_v = (\text{slope}\cdot r + \text{intercept})\bigl(1 + p_1\cos(2\theta_l) + \ldots + \mathbf{p_4}\cos\!\bigl(4(\theta_l-\theta_v)\bigr)\bigr) \qquad \text{\textbf{p₄ lives here — inside log₂(Pᵥ), inside the Gaussian}}$$

---

### Effect 1 — Structural: gradient amplification from being inside the Gaussian

**Gradient for A₂** — A₂ lives in $A_v$, which multiplies $G$ from the outside:
$$\frac{\partial \,\text{pred}}{\partial A_2} = G \cdot \cos(4\theta_l)$$
Since $G \in [0,1]$ always, this is **bounded by 1**. No amplification of any kind.

**Gradient for p₄** — p₄ lives in $P_v$, which appears as $\log_2(P_v)$ inside the Gaussian exponent. Differentiating through the chain $p_4 \to P_v \to \log_2(P_v) \to u \to G \to \text{pred}$:
$$\frac{\partial\, \text{pred}}{\partial p_4} = A_v \cdot G \cdot \underbrace{\frac{-u}{\sigma^2 \ln 2}}_{\text{amplification factor}} \cdot \cos\!\bigl(4(\theta_l - \theta_v)\bigr)$$

The extra factor $|u|/\sigma^2$ comes from:
1. Differentiating $\exp(-u^2/2\sigma^2)$ through $u$ → brings down $-u/\sigma^2$
2. Differentiating $\log_2(P_v)$ through $P_v$ → brings down $1/(P_v \ln 2)$, which simplifies to $1/\ln 2$ after $\partial P_v/\partial p_4$ cancellation

For off-peak stimuli (the majority), $|u| > 0$, amplifying p₄’s gradient beyond A₂’s. **Without L2 normalization, p₄ drifts ~2.5× more than A₂ under null fitting.**

---

### Effect 2 — L2 normalization: amplitude changes become invisible to the loss

The actual loss normalizes predictions before comparing them to data:
$$L = \left\|\frac{\text{pred}}{\|\text{pred}\|} - \frac{\text{target}}{\|\text{target}\|}\right\|^2$$

**What this does to A₂**: A₂ changes $A_v \approx 1 + A_2 \cdot \cos(4\theta) + \ldots$, which is essentially a scalar multiplier on all predictions (an amplitude change). The L2 norm divides this out. As a result:
- Moving A₂ slightly from its initial value changes only the amplitude of predictions
- The norm cancels this → the loss stops responding → gradient collapses toward 0 → A₂ freezes in place

**What this does to p₄**: p₄ shifts which stimulus receives the peak response — a **shape** change (the relative magnitudes of predictions change). Shape changes are preserved through L2 normalization (they alter the pattern of normalized predictions). The loss remains sensitive to p₄ → gradient stays nonzero → p₄ keeps drifting to fit random patterns.

**With L2 normalization, the drift ratio increases from ~2.5× to ~25×.**

---

### The same logic explains the full null distribution figure

| Parameter group | Role | Expected null width |
|---|---|---|
| **A₁, A₂** | Amplitude modulators (outside Gaussian) | **Narrowest**: structural bound + killed by L2 norm |
| **p₁–p₄** | Shape modulators via log₂(Pᵥ) | Moderate: structural $|u|/\sigma^2$ factor + survives L2 norm |
| **intercept, slope** | Baseline frequency (slope scaled by eccentricity $r$) | Wide: same as p-params but $r$ provides extra scaling |
| **σ** | Bandwidth: gradient $\propto u^2/\sigma^3$ | **Widest**: largest amplification factor |

In [ ]:
# ── Synthetic stimuli ──────────────────────────────────────────────────────
torch.manual_seed(42)
np.random.seed(42)
n_stim = 40

theta_l = torch.tensor(np.random.uniform(0, 2*np.pi, n_stim), dtype=torch.float32)
theta_v = torch.tensor(np.random.uniform(0, 2*np.pi, n_stim), dtype=torch.float32)
r_v     = torch.tensor(np.random.uniform(1, 10,      n_stim), dtype=torch.float32)
w_l     = torch.tensor(np.random.uniform(0.5, 8,     n_stim), dtype=torch.float32)


def make_params():
    """Initialize near Broderick parameter values (all p and A params near 0)."""
    return {
        'sigma':     torch.tensor(2.0,   requires_grad=True),
        'slope':     torch.tensor(0.12,  requires_grad=True),
        'intercept': torch.tensor(0.20,  requires_grad=True),
        'p_1':       torch.tensor(0.05,  requires_grad=True),
        'p_2':       torch.tensor(-0.03, requires_grad=True),
        'p_3':       torch.tensor(0.04,  requires_grad=True),
        'p_4':       torch.tensor(0.01,  requires_grad=True),
        'A_1':       torch.tensor(0.04,  requires_grad=True),
        'A_2':       torch.tensor(0.01,  requires_grad=True),
    }


def compute_pred(p):
    """Forward pass: 2D model prediction equations."""
    Av = torch.clamp(
        1 + p['A_1'] * torch.cos(2 * theta_l)
          + p['A_2'] * torch.cos(4 * theta_l),
        min=1e-6)
    Pv = torch.clamp(
        (p['slope'] * r_v + p['intercept']) * (
            1
            + p['p_1'] * torch.cos(2 * theta_l)
            + p['p_2'] * torch.cos(2 * theta_v)
            + p['p_3'] * torch.cos(2 * (theta_l - theta_v))
            + p['p_4'] * torch.cos(4 * (theta_l - theta_v))),
        min=1e-6)
    u = torch.log2(w_l) + torch.log2(Pv)
    return Av * torch.exp(-u**2 / (2 * p['sigma']**2))


def run_null_fitting(n_epochs=2000, use_l2_norm=True, seed=7):
    """Fit model to random (null) target data. Returns parameter trajectories."""
    torch.manual_seed(seed)
    target    = torch.rand(n_stim)
    params    = make_params()
    optimizer = torch.optim.Adam(list(params.values()), lr=1e-3)
    history   = {'A_2': [], 'p_4': []}

    for _ in range(n_epochs):
        optimizer.zero_grad()
        pred = compute_pred(params)
        if use_l2_norm:
            pred_n   = pred   / torch.linalg.norm(pred)
            target_n = target / torch.linalg.norm(target)
            loss = torch.mean((pred_n - target_n) ** 2)
        else:
            loss = torch.mean((pred - target) ** 2)
        loss.backward()
        optimizer.step()
        history['A_2'].append(params['A_2'].item())
        history['p_4'].append(params['p_4'].item())

    return history


# Run both conditions (same random seed and target for fair comparison)
hist_no_norm = run_null_fitting(use_l2_norm=False)
hist_l2_norm = run_null_fitting(use_l2_norm=True)

# ── Plot ────────────────────────────────────────────────────────────────────
epochs = np.arange(2000)

fig, axes = plt.subplots(1, 2, figsize=(11, 4.2))

panel_info = [
    (hist_no_norm, 'Without L2 normalization\n(structural effect only)'),
    (hist_l2_norm, 'With L2 normalization (actual loss)\n(structural + amplitude-blindness)'),
]

for ax, (hist, title) in zip(axes, panel_info):
    ax.plot(epochs, hist['A_2'], color='#e74c3c', lw=2,
            label='A\u2082  (amplitude modulator, outside Gaussian)')
    ax.plot(epochs, hist['p_4'], color='#3498db', lw=2,
            label='p\u2084  (shape modulator, inside log\u2082(P\u1d65))')
    ax.axhline(0.01, color='gray', ls='--', lw=1, alpha=0.7, label='initialization')

    a2_final = hist['A_2'][-1]
    p4_final = hist['p_4'][-1]
    ratio    = abs(p4_final) / max(abs(a2_final), 1e-6)
    ax.set_title(f'{title}\nfinal A\u2082={a2_final:.3f}, p\u2084={p4_final:.3f}  '
                 f'(drift ratio |p\u2084|/|A\u2082| \u2248 {ratio:.0f}\u00d7)')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Parameter value')
    ax.legend(fontsize=9, loc='upper left')

plt.suptitle('Null fitting: A\u2082 vs p\u2084 trajectories when fitting random data',
             fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

## Summary

### Two effects, same direction

| Effect | Mechanism | Contribution |
|---|---|---|
| **Structural** | p₄ is inside log₂(Pᵥ) inside the Gaussian exponent → gradient has extra $|u|/\sigma^2$ factor. A₂ is outside → gradient bounded by $G \leq 1$. | ~2.5× larger drift for p₄ |
| **L2 normalization** | A₂ changes prediction amplitude → divided out by norm → loss becomes flat → gradient collapses → A₂ freezes. p₄ changes prediction shape → survives norm → gradient stays nonzero. | Amplifies total ratio to ~25× |

### Why this explains the full null distribution figure

The same two-effect logic predicts the ordering of all 9 parameters:

- **A₁, A₂** (amplitude modulators, outside Gaussian): amplitude changes killed by L2 norm + bounded structural gradient → **narrowest** null distributions
- **p₁–p₄** (shape modulators via log₂(Pᵥ)): $|u|/\sigma^2$ amplification + shape changes survive L2 norm → **moderate** null distributions
- **intercept, slope** (determine baseline Pᵥ for all stimuli; slope additionally scaled by eccentricity $r$): same as p-params but with larger effective gradient due to $r$ → **wide** null distributions
- **σ** (bandwidth, in the denominator of the Gaussian exponent): gradient $\propto u^2/\sigma^3$, largest amplification factor of all → **widest** null distribution

### Key takeaway

The root cause is that **A₂ modulates amplitude while p₄ modulates shape**. This distinction arises from their structural positions in the model (outside vs. inside the Gaussian), and is then strongly amplified by the L2 normalization in the loss function, which is blind to amplitude but sensitive to shape.